## preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/Kaggle/Copy of princess_corpus.csv')

In [ ]:
data.shape

(7748, 7)

In [ ]:
data.head(2)

,Disney_Period,Text,Speaker_Status,Movie,Speaker,Year,UTTERANCE_NUMBER
0,EARLY,slave in the magic mirror come from the farthe...,NON-P,Snow White,queen,1937,1
1,EARLY,"what wouldst thou know, my queen ?",NON-P,Snow White,mirror,1937,2


In [ ]:
data[data["Movie"]=="Snow White "]

,Disney_Period,Text,Speaker_Status,Movie,Speaker,Year,UTTERANCE_NUMBER
0,EARLY,slave in the magic mirror come from the farthe...,NON-P,Snow White,queen,1937,1
1,EARLY,"what wouldst thou know, my queen ?",NON-P,Snow White,mirror,1937,2
2,EARLY,"magic mirror on the wall, who is the fairest o...",NON-P,Snow White,queen,1937,3
3,EARLY,"famed is thy beauty, majesty. but hold, a love...",NON-P,Snow White,mirror,1937,4
4,EARLY,alas for her ! reveal her name.,NON-P,Snow White,queen,1937,5
...,...,...,...,...,...,...,...
361,EARLY,one song,PRINCE,Snow White,prince,1937,362
362,EARLY,"one song, i have but one song one song, only f...",NON-P,Snow White,chorus,1937,363
363,EARLY,"goodbye. goodbye, grumpy.",PRINCESS,Snow White,snow white,1937,364
364,EARLY,"oh, dopey. goodbye !",NON-P,Snow White,goodbye.,1937,365


In [ ]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words('russian')

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')
from nltk import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def custom_lemmatizer(text):
  return [lemmatizer.lemmatize(w) for w in word_tokenize(text.lower(), language='russian') if w.isalpha() and not w in sw]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
print(*custom_tokenizer(data.Text[0]))
print(*custom_lemmatizer(data.Text[0]))

slav in the magic mirror com from the farthest spac through wind and darkness i summon the speak let me se thy fac
slave in the magic mirror come from the farthest space through wind and darkness i summon thee speak let me see thy face


In [ ]:
index = (("phrase", "lem"), ("phrase", "stem"), ("sentence", "lem"), ("sentence", "stem"))
train_res = pd.DataFrame(index=index,
                  columns=["countvec", "tf-idf", "hashing"])

In [ ]:
test_res = pd.DataFrame(index=index,
                        columns=["countvec", "tf-idf", "hashing"])

In [ ]:
index = (("phrase", "lem", "train"), ("phrase", "lem", "test"), ("phrase", "stem", "train"), ("phrase", "stem", "test"), 
         ("sentence", "lem", "train"), ("sentence", "lem", "test"), ("sentence", "stem", "train"), ("sentence", "stem", "train"))
res = pd.DataFrame(index=index,
                  columns=["countvec", "tf-idf", "hashing"])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data["Text"], data["Speaker_Status"],  test_size = 0.3, random_state=0)

## Count Vectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cntvec = CountVectorizer(tokenizer=custom_tokenizer)
train_cntvec = cntvec.fit_transform(X_train)
test_cntvec = cntvec.transform(X_test)

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(random_state=0)
clf.fit(train_cntvec, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
from sklearn.metrics import accuracy_score

train_res.loc["phrase", "stem"].at["countvec"] = np.round(accuracy_score(clf.predict(train_cntvec), y_train), decimals=3)

test_res.loc["phrase", "stem"].at["countvec"] = np.round(accuracy_score(clf.predict(test_cntvec), y_test), decimals=3)

In [ ]:
cntvec1 = CountVectorizer(tokenizer=custom_lemmatizer)
train_cntvec = cntvec1.fit_transform(X_train)
test_cntvec = cntvec1.transform(X_test)

clf = SGDClassifier(random_state=0)
clf.fit(train_cntvec, y_train)

train_res.loc["phrase", "lem"].at["countvec"] = np.round(accuracy_score(clf.predict(train_cntvec), y_train), decimals=3)
test_res.loc["phrase", "lem"].at["countvec"] = np.round(accuracy_score(clf.predict(test_cntvec), y_test), decimals=3)

In [ ]:
cntvec3 = CountVectorizer(tokenizer=custom_tokenizer)
train_cntvec = cntvec3.fit_transform(X_train)
test_cntvec = cntvec3.transform(X_test)

clf = SGDClassifier(random_state=0)
clf.fit(train_cntvec, y_train)

train_res.loc["phrase", "lem"].at["countvec"] = np.round(accuracy_score(clf.predict(train_cntvec), y_train), decimals=3)
test_res.loc["phrase", "lem"].at["countvec"] = np.round(accuracy_score(clf.predict(test_cntvec), y_test), decimals=3)

## TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer=custom_tokenizer, max_df=.9)
train_tfidf = tfidf.fit_transform(X_train)
test_tfidf = tfidf.transform(X_test)

In [ ]:
clf1 = SGDClassifier(random_state=0)
clf1.fit(train_tfidf, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
train_res.loc["phrase", "stem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(train_tfidf), y_train), decimals=3)

test_res.loc["phrase", "stem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(test_tfidf), y_test), decimals=3)

In [ ]:
tfidf1 = TfidfVectorizer(tokenizer=custom_lemmatizer, max_df=.9)
train_tfidf = tfidf1.fit_transform(X_train)
test_tfidf = tfidf1.transform(X_test)

clf1 = SGDClassifier(random_state=0)
clf1.fit(train_tfidf, y_train)

train_res.loc["phrase", "lem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(train_tfidf), y_train), decimals=3)
test_res.loc["phrase", "lem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(test_tfidf), y_test), decimals=3)

## Hashing

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

hash_vec = HashingVectorizer(tokenizer=custom_tokenizer, n_features=10000)
train_hash = hash_vec.fit_transform(X_train)
test_hash = hash_vec.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
clf2 = SGDClassifier(random_state=0)
clf2.fit(train_hash, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
train_res.loc["phrase", "stem"].at["hashing"] = np.round(accuracy_score(clf2.predict(train_hash), y_train), decimals=3)

test_res.loc["phrase", "stem"].at["hashing"] = np.round(accuracy_score(clf2.predict(test_hash), y_test), decimals=3)

In [ ]:
hash_vec1 = HashingVectorizer(tokenizer=custom_lemmatizer, n_features=10000)
train_hash = hash_vec1.fit_transform(X_train)
test_hash = hash_vec1.transform(X_test)

clf2 = SGDClassifier(random_state=0)
clf2.fit(train_hash, y_train)

train_res.loc["phrase", "lem"].at["hashing"] = np.round(accuracy_score(clf2.predict(train_hash), y_train), decimals=3)
test_res.loc["phrase", "lem"].at["hashing"] = np.round(accuracy_score(clf2.predict(test_hash), y_test), decimals=3)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
train_res

countvec tf-idf hashing
phrase   lem     0.848  0.804   0.751
         stem    0.846  0.806   0.746
sentence lem       NaN    NaN     NaN
         stem      NaN    NaN     NaN

In [ ]:
test_res

countvec tf-idf hashing
phrase   lem     0.622  0.651   0.657
         stem     0.62  0.652   0.656
sentence lem       NaN    NaN     NaN
         stem      NaN    NaN     NaN

# По предложениям

In [ ]:
import nltk

In [ ]:
sents = sum([nltk.sent_tokenize(x) for x in data.Text], [])

In [ ]:
len(sents)

13823

In [ ]:
sents[:10]

['slave in the magic mirror come from the farthest space through wind and darkness i summon thee.',
 'speak !',
 'let me see thy face.',
 'what wouldst thou know, my queen ?',
 'magic mirror on the wall, who is the fairest one of all ?',
 'famed is thy beauty, majesty.',
 'but hold, a lovely maid i see.',
 'rags cannot hide her gentle grace.',
 'alas, she is more fair than thee.',
 'alas for her !']

In [ ]:
data["n_sent"] = data["Text"].apply(lambda x: len(nltk.sent_tokenize(x)))
data["target"] = data.apply(lambda x: [x.Speaker_Status]*x.n_sent, axis=1)

In [ ]:

target = sum(data["target"].tolist(), [])

In [ ]:
len(target)

13823

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sents, target,  test_size = 0.3, random_state=0)

In [ ]:
cntvec2 = CountVectorizer(tokenizer=custom_tokenizer)
train_cntvec = cntvec2.fit_transform(X_train)
test_cntvec = cntvec2.transform(X_test)

clf = SGDClassifier(random_state=0)
clf.fit(train_cntvec, y_train)

train_res.loc["sentence", "stem"].at["countvec"] = np.round(accuracy_score(clf.predict(train_cntvec), y_train), decimals=3)
test_res.loc["sentence", "stem"].at["countvec"] = np.round(accuracy_score(clf.predict(test_cntvec), y_test), decimals=3)

In [ ]:
cntvec3 = CountVectorizer(tokenizer=custom_tokenizer)
train_cntvec = cntvec3.fit_transform(X_train)
test_cntvec = cntvec3.transform(X_test)

clf = SGDClassifier(random_state=0)
clf.fit(train_cntvec, y_train)

train_res.loc["sentence", "lem"].at["countvec"] = np.round(accuracy_score(clf.predict(train_cntvec), y_train), decimals=3)
test_res.loc["sentence", "lem"].at["countvec"] = np.round(accuracy_score(clf.predict(test_cntvec), y_test), decimals=3)

In [ ]:
tfidf2 = TfidfVectorizer(tokenizer=custom_tokenizer, max_df=.9)
train_tfidf = tfidf2.fit_transform(X_train)
test_tfidf = tfidf2.transform(X_test)

clf1 = SGDClassifier(random_state=0)
clf1.fit(train_tfidf, y_train)

train_res.loc["sentence", "stem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(train_tfidf), y_train), decimals=3)
test_res.loc["sentence", "stem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(test_tfidf), y_test), decimals=3)

In [ ]:
tfidf3 = TfidfVectorizer(tokenizer=custom_lemmatizer, max_df=.9)
train_tfidf = tfidf3.fit_transform(X_train)
test_tfidf = tfidf3.transform(X_test)

clf1 = SGDClassifier(random_state=0)
clf1.fit(train_tfidf, y_train)

train_res.loc["sentence", "lem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(train_tfidf), y_train), decimals=3)
test_res.loc["sentence", "lem"].at["tf-idf"] = np.round(accuracy_score(clf1.predict(test_tfidf), y_test), decimals=3)

In [ ]:
hash_vec2 = HashingVectorizer(tokenizer=custom_tokenizer, n_features=10000)
train_hash = hash_vec2.fit_transform(X_train)
test_hash = hash_vec2.transform(X_test)

clf2 = SGDClassifier(random_state=0)
clf2.fit(train_hash, y_train)

train_res.loc["sentence", "stem"].at["hashing"] = np.round(accuracy_score(clf2.predict(train_hash), y_train), decimals=3)
test_res.loc["sentence", "stem"].at["hashing"] = np.round(accuracy_score(clf2.predict(test_hash), y_test), decimals=3)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
hash_vec3 = HashingVectorizer(tokenizer=custom_lemmatizer, n_features=10000)
train_hash = hash_vec3.fit_transform(X_train)
test_hash = hash_vec3.transform(X_test)

clf2 = SGDClassifier(random_state=0)
clf2.fit(train_hash, y_train)

train_res.loc["sentence", "lem"].at["hashing"] = np.round(accuracy_score(clf2.predict(train_hash), y_train), decimals=3)
test_res.loc["sentence", "lem"].at["hashing"] = np.round(accuracy_score(clf2.predict(test_hash), y_test), decimals=3)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [ ]:
train_res

countvec tf-idf hashing
phrase   lem     0.848  0.804   0.751
         stem    0.846  0.806   0.746
sentence lem     0.771  0.724   0.688
         stem    0.771  0.726   0.693

In [ ]:
test_res

countvec tf-idf hashing
phrase   lem     0.622  0.651   0.657
         stem     0.62  0.652   0.656
sentence lem     0.651  0.665   0.655
         stem    0.651  0.661   0.663

In [ ]:
clf2.predict(train_hash)

array(['NON-P', 'PRINCESS', 'NON-P', ..., 'NON-P', 'NON-P', 'PRINCESS'],
      dtype='<U8')